In [1]:
import pandas as pd

# Automatically clean section titles mistakenly included in text body

In [69]:
def locate_section_title(text):
    '''Helper function to split a text body where part of the section title is mistakenly
       included in the text body
       
        Args:
           text: text body to split
        Returns:
            (section_title, corrected_text) tuple
    '''
    # Counter for block of 4 contiguous letters to identify end of section title
    year_counter = 0
    
    for ind, char in enumerate(text):
        if str.isdigit(char):
            year_counter += 1
            
            # If 4 contiguous digits found, we've located where to split the string
            if year_counter == 4:
                # Check for edge case of /digit
                if text[ind + 1] == '/' and str.isdigit(text[ind + 2]):
                    return text[:ind+3], text[ind+3:].strip()
                else:
                    return text[:ind+1], text[ind+1:].strip()
        else:
            year_counter = 0
        
    # No splitting point found
    return None, None

In [102]:
# Load csv file to clean
df = pd.read_csv('csv/segmented_manually_cleaned.csv')

In [103]:
# Apply splitting function to fix section titles and text
for ind, row in df.iterrows():
    if len(row['section_title'].strip()) < 5:
        # Apply splitting function
        title_segment, text = locate_section_title(row['text'])
        
        if title_segment is not None:
            if row['section_title'].strip()[-1] == '.':
                df.loc[ind]['section_title'] += ' ' + title_segment
            elif str.isdigit(row['section_title'].strip()[-1]):
                df.loc[ind]['section_title'] += '. ' + title_segment
            df.loc[ind]['text'] = text

In [104]:
df.to_csv('csv/segmented_cleaned.csv', index=False)